# Working with Continuous Data

We have previously done some work with continuous data when we were learning about visualizations. In this section we are going to look at some methods that will help you examine associations between continuous variables. The primary tool we are going to use is the correlation. 

In this section, we will use a new library for visualizations, called `seaborn`. Seaborn is traditionally imported with the abbreviation `sns`

The specific methods we will use that we haven't covered before are:

* `corr()`, a dataframe method from pandas
* `heatmap()`, a plotting method from seaborn
* `pairplot()`, a plotting method from seaborn
* `unique()` a series method from pandas



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# For slightly nicer charts
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 150
sns.set(style="ticks")

***

## Preparing our Dataset

For this section we will use a selection from the dataset `fandango_score_comparison.csv` ([source](https://github.com/fivethirtyeight/data/tree/master/fandango)), containing data about movie ratings from Fandango, Rotten Tomatoes, IMDB, and Metacritic. The dataset was used for this article, [Be Suspicious Of Online Movie Ratings, Especially Fandango’s](https://fivethirtyeight.com/features/fandango-movies-ratings/), published by [fivethirtyeight](https://fivethirtyeight.com/).

First, we read a CSV file of the movie ratings and assign it to a variable called `df_full`. We will then select a subset of the data and assign the subsection to the variable `df`.
Each row in the dataset represents one movie.  The columns are labelled as follows: 

* FILM: The film in question
* RottenTomatoes: The Rotten Tomatoes Tomatometer score for the film
* RottenTomatoes_User: The Rotten Tomatoes user score for the film
* Metacritic: The Metacritic critic score for the film
* Metacritic_User: The Metacritic user score for the film
* IMDB: The IMDb user score for the film
* Fandango_Stars: The number of stars the film had on its Fandango movie page
* Fandango_Ratingvalue: The Fandango ratingValue for the film, as pulled from the HTML of each page. This is the actual average score the movie obtained.



In [ ]:
df_full = pd.read_csv("fandango_score_comparison.csv")
df = df_full.loc[:,'FILM':'Fandango_Ratingvalue'] # this creates a subset of the dataframe using location based indexing, which we will discuss later 
df.head()

In [ ]:
df.info()

Our dataset has 146 movies (one per row) and eight columns. The first column labelled `'FILM'` contains strings with the name of the Film and its year of release. The remaining columns are either integers or floats representing various kinds of ratings of the quality of the film.

***

Before we start looking at how the ratings are related to one another, let's look at some descriptive statistics.

In [ ]:
df.describe().transpose()

Notice, the variations in the mean, min, and max values across the different ratings sites. Some of the ratings systems appear to be using 100 points scales, others 10 points scales, and others 5 points scales. Also, no matter what the scale, the average rating tends to be just a bit higher than the mid-point of the scale. 

The different scales make looking at the descriptive statistics a bit confusing so let's convert all the scales to be the same. This conversion process is commonly called 'normalizing' your data. In this case we are going to convert everything to a 10-point scale. 

We are NOT going to 'copy over' the existing values. Instead we are going to calculate new values and assign them to new columns.  

First, we will normalize the 100-point columns and assign them to new columns. 

In [ ]:
hundred_list = ['RottenTomatoes', 'RottenTomatoes_User', 'Metacritic']
df[['RottenTomatoes_Norm', 'RottenTomatoes_User_Norm', 'Metacritic_Norm']] = df[hundred_list]/10
df.head()

Second, we will normalize the 5-point columns and assign them to new columns. 

In [ ]:
five_list = ['Fandango_Stars', 'Fandango_Ratingvalue']
df[['Fandango_Stars_Norm', 'Fandango_Ratingvalue_Norm']] = df[five_list]*2
df.head()

Now we are going to drop all of the non-normed columns.

In [ ]:
non_normed_ratings_list = ['RottenTomatoes', 'RottenTomatoes_User', 'Metacritic', 'Fandango_Stars', 'Fandango_Ratingvalue']
df = df.drop(columns=non_normed_ratings_list)
df.head()

## A Small Aside About Reusable Code

So our output above looks great but our column order is bit jumbled. Let's do a quick fix to get the ratings in alphabetical order. We are going to need a list with the labels in the proper order first. We can make a list manually:

In [ ]:
label_order_list_manual = ['FILM', 'Fandango_Ratingvalue_Norm', 'Fandango_Stars_Norm', 'IMDB', 'Metacritic_Norm', 'Metacritic_User', 'RottenTomatoes_Norm', 'RottenTomatoes_User_Norm']

Or we can make a list using a few properties and methods.

In [ ]:
label_order_list_fancy = df.columns.tolist()
label_order_list_fancy.sort() #Note that the sort() method sorts the list 'in place' and does not return anything, therefore it cannot be assigned to a variable

Both methods produce the same results.

In [ ]:
print('Manual result:', label_order_list_manual)
print('Fancy  result:', label_order_list_fancy)

For small datasets, manually creating lists may make sense and may be an efficient use of your time. However, it will often be the case that writing code to automate these types of processes will save you time and make your process less error prone. This will be particularly true for large datasets. Another advatage of solving your problems with code is the code can often be reused. The code for the 'fancy' approach above could be used with any dataset, while the manually created list will likely only ever be useful this one specific time. 

So we now have our list (we can use either), let's use it to 'reorder' our dataframe.

In [ ]:
df = df[label_order_list_fancy]
df.head()

That looks pretty good. Let's rerun our descriptives and take another look at the variables.

In [ ]:
df.describe().transpose()

Now that we have 'normed' the data the difference in mean ratings across the rating sites really stands out. (For more about this issue see [the original article](https://fivethirtyeight.com/features/fandango-movies-ratings/)). However, what we are interested in right now is the degree to which the various ratings are associated (or how much they covary). 
***

## Looking at Associations Using Correlations
We often want to know the degree to which one variable is associated with another. We don't always say 'associated' when asking these types of questions; instead, we will often use the word 'predict'. For example, we might say we want to know if the price of milk will 'predict' the price of cheese. Unfortunately, the word 'predict' (or the phrase 'this predicts that') often implies things we (or our data) are not prepared to address. So it's a bit more cautious to talk about whether or not the price of milk is 'associated' with the price of cheese.

If you are interested in how two variable are associated there are a bunch of formal statistical tests you might use to characterize the association. The type of test you would use depends on the type of data you are trying to associate. The most common case involves characterizing the linear association between two continuous variables. The test we are going to use to do this is formally called [Pearson's correlation coefficient, Pearson's r, or a bivariate correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient). You might be tempted just to call it a correlation, and people often do this; but there are other types of correlational tests that are more appropriate for other types of data so its better to be specific about what type of correlational test you are using. 

Pearson's r has values between -1 and +1, with -1 denoting a perfect negative association (as one variable increases the other decreases). +1 denotes a perfect positive association (as one variable increase the other increases to the same degree). 0 denotes no association at all between the two variables.

There is a built in method for looking at correlations in Pandas. If we index a single column from the dataframe, and then pass a second column as an argument to the `.corr()` method it will return a single float value. By default, `.corr()` returns Pearson's r; however, you could use a parameter to specify other types of correlations. 

Let's look at the association between movie ratings from Metacritic users and from RottenTomatoes_Users.

In [ ]:
df['Metacritic_User'].corr(df['RottenTomatoes_User_Norm'])

Rating are positively correlated, and the association is fairly strong but isn't perfect. Now would be a good time for a scatterplot to get a better sense of what this r value means. 

In [ ]:
plt.scatter(x='Metacritic_User', y='RottenTomatoes_User_Norm', data=df)
plt.xlabel("Metacritic Rating")
plt.ylabel("RT Rating")
plt.title("Metacritic vs RT Ratings")
plt.show()

So an r-value of .68 means that we generally see that movies scoring high on one site are also scoring high on the other site. BUT, there are a some exceptions. A few movies that are rated below a five on metacritic but above a seven on Rotten Tomatoes. 

Just for fun, let's check into that. 

In [ ]:
df_weird = df.query(" (Metacritic_User < 5) and (RottenTomatoes_User_Norm > 7) ")
df_weird.head()

Now there's some strange data. See if you can come up with an idea for why these films ratings would be so inconsistent. Poking around in df_full might help you out here. 

Back to the scatter plot above... It looks like there might be a stronger association between the two ratings for movies that are highly rated and less of an association for movies that are not as highly rated. There are some formal statistical tests for this type of situation, but for now let's just look at it with the tools we have. 

Let's select only the movies that are rated less than 5 by metacritic and then rerun our correlation. 

In [ ]:
df_meta5_low = df.query('Metacritic_User < 5 ')
df_meta5_low['Metacritic_User'].corr(df_meta5_low['RottenTomatoes_User_Norm'])

There does appear to be less agreement. Just to make a more apples to apple comparison, we can rerun this looking at films scoring greater than 5 on Metacritic. 

In [ ]:
df_meta5_high = df.query(" Metacritic_User > 5 ")
df_meta5_high['Metacritic_User'].corr(df_meta5_high['RottenTomatoes_User_Norm'])

Its notable that while there is a stronger assocation between ratings at the high end of the scale, using the full range of movies actually showed the strongest association between the ratings. 
***


If we call the corr() method on the entire dataframe, pandas will calculate correlations for all columns with numeric datatypes. 

In [ ]:
df.corr()

The output above is often referred to as a correlation table. This correlation table allows you to see how all of our movie rating variables are correlated with one another. 
Notice there is a diagonal of 1.000 values in this output. This is the variables correlated with themselves, which always results perfect positive correlations. It is also worth noting that the left and right sides of that diagonal are redundant mirror-images of one another. Since both the diagonal values and the mirror image values provide no real information, you will sometimes see these values replaced with some other kind of statistic that provides other information. We will see an example of this in just a minute. 

Giant tables of correlation values are often challenging to interpret on thier own. One way to help is to generate a visualization of the table itself. We can do this using something called a heatmap. 
In a heat map colors are used to represent different values.

To make a heatmap we are going to import a new library called `seaborn`. You can think of `seaborn` as an extension of `matplotlib`. It does similar things in similar ways, but also makes some things easier to do. We are going to call the heatmap method from the seaborn library and we are going to pass it the correlation table above. We are also going to include the argument `cmap='Blues'` to control the type of colors that are displayed in the heatmap.

In [ ]:
correlation_table = df.corr()
sns.heatmap(correlation_table, cmap="Blues")

Another method for visualizing associations across a data set is a scatter matrix. A scatter matrix is a set of scatterplots that visualize the information you would find in a correlation table.
We are going to use the seaborn `pairplot()` method and pass it our dataframe to create our scatter matrix of movie ratings.

In [ ]:
sns.pairplot(df) # this code creates the scatter matrix

There's a lot of information packed into the scatter matrix. First, notice the diagonals. Rather than showing us a perfect (and uninformative) correlation this scatter matrix if showing us histograms for each variable. From the histograms we wan see that 'fandango_starred_norm' only has discreet integer values. The histograms also show us that the Fandango ratings and the Rotten Tomato ratings tend to be skewed toward the positive side of the distribution. 

*** 
## Looking at Associations by Group (Category)
We will sometimes want to know if associations vary by group. For example, if our movie rating dataset had movie genre data we might be interested in finding out if the ratings from different sites more strongly associated for dramas than for comedies. Or if we had multiple years of data we might be interested to see if the associations changed by year. 

With some slight modifications to the tools we have already discussed we can start looking at associations by group. To do this we are going to use a new dataset, `'recent-grads.csv'`, which contains data about employment and salaries for recent college graduates. The data comes from [here](https://github.com/fivethirtyeight/data/tree/master/college-majors) and was used for the story [The Economic Guide to Picking Your Major](https://fivethirtyeight.com/features/the-economic-guide-to-picking-a-college-major/), published by [fivethirtyeight](https://fivethirtyeight.com/).



In [ ]:
df_grad_full = pd.read_csv("recent-grads.csv") # here we read the full dataset
df_grad = df_grad_full[['Major_category', 'Major', 'ShareWomen', 'Unemployment_rate','Median']] # here we use label based indexing to create a dataset that only contains a few columns of interest
df_grad.head()

In [ ]:
df_grad.info()

There are 173 rows in the dataframe. Each row represents a different major. The majors are grouped into one of 16 categories. We can see the categories by calling the `unique()` method on the `'Major_category'` column.

In [ ]:
df_grad['Major_category'].unique()

The remaining categories represent the following: 
* ShareWomen: Percentage of graduates classified as women
* Unemployment_rate: Rate of unemployment
* Median: Median earnings of full-time, year-round workers

If we use `corr()` to look at our dataframe, one result immediately stands out. 

In [ ]:
df_grad.corr()

We can use a scatter matrix to further examine this association. This time we are only going to look at associations for two different categories of majors.. First, we will use query to make a dataset that only contains categories that we are interested in. 

In [ ]:
df_grad_sub = df_grad.query(' Major_category == "Engineering" or Major_category == "Psychology & Social Work" or Major_category == "Business"')

Now we are going to use the seaborn `pairplot()` method again, but with one small addition. We will to pass the argument `'Major_category'` to the parameter `hue`. This parameter produces pairplots with groups. 

In [ ]:
sns.pairplot(df_grad_sub, hue='Major_category')

The diagonals are again providing information about the frequency of specific values, but in this case the frequencies are represented in different colors for the different categories of majors. The same is done for the scatterplots. 